In [5]:
from math import exp 

from pyneuroml.analysis.ChannelHelper import evaluate_HHExpRate, evaluate_HHExpLinearRate, evaluate_HHSigmoidRate


In [7]:
v = 0 # mV 

rate_alpha_m0 = 0.32 *(v+54)     / (1- exp(-1*(v+54)/4.))         # From chapter eqn 4
rate_alpha_m = (1.28 *(v - -54)/4.) / (1- exp(-1*(v- -54)/4.))    # Restructured

print('%s per_ms'%rate_alpha_m0)
print('%s per_ms'%rate_alpha_m)
evaluate_HHExpLinearRate("1.28per_ms", "-54mV", "4mV",'%smV'%v)

17.280023690205493 per_ms
17.280023690205493 per_ms


In [8]:
evaluate_HHExpLinearRate("1.28per_ms", "-54mV", "4mV",'%smV'%v)

In [41]:
import math
from neuroml import NeuroMLDocument
from neuroml import Cell
from neuroml import IonChannelHH
from neuroml import GateHHRates
from neuroml import HHRate
from neuroml import Q10ConductanceScaling
from neuroml import Q10Settings
from neuroml import BiophysicalProperties
from neuroml import MembraneProperties
from neuroml import IncludeType
from neuroml import ChannelDensity
from neuroml import SpikeThresh
from neuroml import SpecificCapacitance
from neuroml import InitMembPotential
from neuroml import IntracellularProperties
from neuroml import Resistivity
from neuroml import Resistivity
from neuroml import Morphology, Segment, Point3DWithDiam
from pyneuroml import pynml
from pyneuroml.lems import LEMSSimulation
import os
from CellBuilder import add_channel_density

In [15]:
print(os.getcwd())

/Users/shayan/Desktop/CN/gsoc_cn/BahlEtAl2012/NeuroML2


In [51]:
def create_kfast_channel():
    """Create the kfast channel.

    This will create the kfast channel and save it to a file.
    It will also validate this file.

    returns: name of the created file
    """
    kfast_channel = IonChannelHH(id="kfast_channel", notes="K fast channel for HH cell", conductance="10pS", species="k")
    gate_n = GateHHRates(id="n", instances="1", notes="n gate for kfast channel")

    m_forward_rate = HHRate(type="HHExpLinearRate", rate="1per_ms", midpoint="-40mV", scale="10mV")
    m_reverse_rate = HHRate(type="HHExpRate", rate="4per_ms", midpoint="-65mV", scale="-18mV")

    conductance_scaling = Q10ConductanceScaling(experimental_temp="23degC", q10_factor=2.3)
    
    kfast_channel.q10_conductance_scalings.append(conductance_scaling)

    gate_n.q10_settings = Q10Settings(type="q10ExpTemp", q10Factor=2.3, experimentalTemp="23degC")
    gate_n.forward_rate = m_forward_rate
    gate_n.reverse_rate = m_reverse_rate
    kfast_channel.gate_hh_rates.append(gate_n)

    kfast_channel_doc = NeuroMLDocument(id="kfast_channel", notes="Kfast channel for HH neuron")
    kfast_channel_fn = "HH_example_kfast_channel.nml"
    kfast_channel_doc.ion_channel_hhs.append(kfast_channel)

    pynml.write_neuroml2_file(nml2_doc=kfast_channel_doc, nml2_file_name=kfast_channel_fn, validate=True)

    return kfast_channel_fn

In [ ]:
def create_kslow_channel():
    

In [57]:
from neuroml import GateHHTauInf
from neuroml import Q

nap_channel = IonChannelHH(id="nap_channel", notes="Nap channel for HH cell", conductance="10pS", species="na")
gate_n = GateHHRates(id="m", instances="1", notes="m gate for Nap channel")

#def create_nap_channel():
    

In [83]:
gate_n.q10_settings = Q10Settings(type="q10ExpTemp", q10Factor=2.3, experimentalTemp="23degC")

In [84]:
gate_n.

AttributeError: 'GateHHRates' object has no attribute 'factor'

In [ ]:
def create_nat_channel():

In [ ]:
def create_km_channel():

In [72]:
pyr_cell_doc = NeuroMLDocument(id='cell', notes="Layer 5 Pyramidal cell")
pyr_cell_doc.includes.append(IncludeType("nap.channel.nml"))
pyr_cell_doc.includes.append(IncludeType("kfast.channel.nml"))
# print(pyr_cell_doc)
ChannelDensity?


Init signature:
ChannelDensity(
    neuro_lex_id=None,
    id=None,
    ion_channel=None,
    cond_density=None,
    erev=None,
    segment_groups='all',
    segments=None,
    ion=None,
    variable_parameters=None,
    extensiontype_=None,
    gds_collector_=None,
    **kwargs_,
)
Docstring:     
ChannelDensity -- Specifies a time varying ohmic conductance density, **gDensity,**  which is distributed on an area of the **cell**  ( specified in  **membraneProperties**  ) with fixed reversal potential **erev**  producing a current density **iDensity**


:param erev: The reversal potential of the current produced
:type erev: voltage
:param condDensity:
:type condDensity: conductanceDensity
File:           ~/opt/anaconda3/envs/neuron/lib/python3.10/site-packages/neuroml/nml/nml.py
Type:           type
Subclasses:     ChannelDensityVShift


In [87]:
def create_cell():
    pyr_cell_doc = NeuroMLDocument(id='cell', notes="Layer 5 Pyramidal cell")
    pyr_cell_fn = "pyr5_cell.nml"
    print(os.getcwd())
    pyr_cell_doc.includes.append(IncludeType("kfast.channel.nml"))
    pyr_cell_doc.includes.append(IncludeType("kslow.channel.nml"))
    pyr_cell_doc.includes.append(IncludeType("nat.channel.nml"))
    pyr_cell_doc.includes.append(IncludeType("nap.channel.nml"))
    pyr_cell_doc.includes.append(IncludeType("IKM.nml"))

    # Define a cell
    pyr_cell = Cell(id="pyr_cell", notes="A single compartment Layer 5 Pyramidal cell")

    # Define its biophysical properties
    bio_prop = BiophysicalProperties(id="pyr_b_prop")
    #  notes="Biophysical properties for Layer 5 Pyramidal cell")

    # Membrane properties are a type of biophysical properties
    mem_prop = MembraneProperties()
    # Add membrane properties to the biophysical properties
    bio_prop.membrane_properties = mem_prop

    # Append to cell
    pyr_cell.biophysical_properties = bio_prop

    # Channel density for kfast channel
    kfast_channel_density = ChannelDensity(id="kfast_channel", cond_density="67.2 S_per_m2", erev="-80.39 mV", ion="k", ion_channel="kfast")
    mem_prop.channel_densities.append(kfast_channel_density)

    kslow_channel_density = ChannelDensity(id="kslow_channels", cond_density="475.82 S_per_m2", erev="-80.39 mV", ion="kslow", ion_channel="kslow")
    mem_prop.channel_densities.append(kslow_channel_density)

    nat_channel_density = ChannelDensity(id="nat_channel", cond_density="236.62 S_per_m2", erev="-80.39 mV", ion="nat", ion_channel="nat")
    mem_prop.channel_densities.append(nat_channel_density)

    nap_channel_density = ChannelDensity(id="nap_channels", cond_density="1.44 S_per_m2", erev="-80.39 mV", ion="nap", ion_channel="nap")
    mem_prop.channel_densities.append(nap_channel_density)

    km_channel_density = ChannelDensity(id="km_channels", cond_density="475.82 S_per_m2", erev="-80.39 mV", ion="km", ion_channel="km")
    mem_prop.channel_densities.append(km_channel_density)

    # add_channel_density(pyr_cell, pyr_cell_doc,
    #                 cd_id="kfast_soma",
    #                 cond_density="67.2 S_per_m2",
    #                 ion_channel="kfast_channel",
    #                 ion_chan_def_file="kfast.channel.nml",
    #                 erev="-80.39mV",
    #                 ion="kfast")

    # Other membrane properties
    mem_prop.spike_threshes.append(SpikeThresh(value="-20mV"))
    mem_prop.specific_capacitances.append(SpecificCapacitance(value="2.23 uF_per_cm2"))
    mem_prop.init_memb_potentials.append(InitMembPotential(value="-65mV"))

    intra_prop = IntracellularProperties()
    intra_prop.resistivities.append(Resistivity(value="0.1 kohm_cm"))

    # Add to biological properties
    bio_prop.intracellular_properties = intra_prop

    # Morphology
    morph = Morphology(id="pyr_cell_morph")
    #  notes="Simple morphology for the HH cell")
    seg = Segment(id="0", name="soma", notes="Soma segment")
    # We want a diameter such that area is 1000 micro meter^2
    # surface area of a sphere is 4pi r^2 = 4pi diam^2
    diam = math.sqrt(1682 / math.pi)
    proximal = distal = Point3DWithDiam(x="0", y="0", z="0", diameter=str(diam))
    seg.proximal = proximal
    seg.distal = distal
    morph.segments.append(seg)
    pyr_cell.morphology = morph

    pyr_cell_doc.cells.append(pyr_cell)
    pynml.write_neuroml2_file(nml2_doc=pyr_cell_doc, nml2_file_name=pyr_cell_fn, validate=True)
    return pyr_cell_fn

create_cell()


pyNeuroML >>> INFO - Executing: (java -Xmx400M  -jar  "/Users/shayan/opt/anaconda3/envs/neuron/lib/python3.10/site-packages/pyneuroml/lib/jNeuroML-0.11.1-jar-with-dependencies.jar" -validate "pyr5_cell.nml" ) in directory: .


/Users/shayan/Desktop/CN/gsoc_cn/BahlEtAl2012/NeuroML2


pyNeuroML >>> INFO - Command completed. Output: 
  jNeuroML >>   jNeuroML v0.11.1
  jNeuroML >>  Validating: /Users/shayan/Desktop/CN/gsoc_cn/BahlEtAl2012/NeuroML2/pyr5_cell.nml
  jNeuroML >>  Valid against schema and all tests
  jNeuroML >>  No warnings
  jNeuroML >>  
  jNeuroML >>  Validated 1 files: All valid and no warnings
  jNeuroML >>  
  jNeuroML >>  


'pyr5_cell.nml'

    <ComponentType name="kslow_b_tau" extends="baseVoltageDepTime">
        <Constant name="TIME_SCALE" dimension="time" value="1ms"/>
        <Constant name="VOLT_SCALE" dimension="voltage" value="1mV"/>
        <Constant name="v05b" dimension="none" value="-58"/>
        <Constant name="b0" dimension="none" value="360"/>
        <Constant name="b11" dimension="none" value="1010"/>
        <Constant name="b2" dimension="none" value="-75"/>
        <Constant name="b3" dimension="none" value="48"/>
        <Constant name="b4" dimension="none" value="23.7"/>
        <Constant name="b5" dimension="none" value="-54"/>

        <Dynamics>
            <DerivedVariable name="V" dimension="none" value="v / VOLT_SCALE"/>
            <DerivedVariable name="t" exposure="t" dimension="time" value="(b0+(b11+b4*(V-b5))*exp(-(V-b2)*(V-b2)/(b3*b3))) * TIME_SCALE "/>
        </Dynamics>

    </ComponentType>


    <ComponentType name="kslow_b1_tau" extends="baseVoltageDepTime">
        <Constant name="TIME_SCALE" dimension="time" value="1ms"/>
        <Constant name="VOLT_SCALE" dimension="voltage" value="1mV"/>
        <Constant name="bb0" dimension="none" value="2350"/>
        <Constant name="bb1" dimension="none" value="1380"/>
        <Constant name="bb2" dimension="none" value="0.01118"/>
        <Constant name="bb3" dimension="none" value="-210"/>
        <Constant name="bb4" dimension="none" value="0.0306"/>


        <Dynamics>
            <DerivedVariable name="V" dimension="none" value="v / VOLT_SCALE"/>
            <DerivedVariable name="t" exposure="t" dimension="time" value="(bb0+bb1*exp(-bb2*V)+bb3*exp(-bb4*V)) * TIME_SCALE "/>
        </Dynamics>

In [106]:
from neuroml import GateFractional
from neuroml import GateFractionalSubgate

gate_n = GateFractional(id="n", instances="1", notes="n gate for kfast channel")
gate_n.sub_gates.append(GateFractionalSubgate(id="bb", fractional_conductance="0.5"))

# gate_n.sub_gates(id="n1", instances="0.5")
# gate_n.sub_gates(id="n2", instances="0.5")
# print(GateFractional)
gate_n
#    gate_n = GateHHRates(id="n", instances="1", notes="n gate for kfast channel")